In [3]:
"""
This script computes the optimal SE(3) transformation between corresponding 2D and 3D points using non-linear optimiziation.
"""
import numpy as np
from numpy.typing import NDArray
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [4]:
# Define the 3D points in world coordinates
points_3d = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1], [1, 1, 1]])

# Define the 2D points in image coordinates
points_2d = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])

# Define the camera intrinsic matrix
K = np.array([[1.0, 0.0, 0.5], [0.0, 1.0, 0.5], [0.0, 0.0, 1.0]])

# Define the initial camera extrinsic parameters
R = np.eye(3)
t = np.zeros(3)

In [5]:
K = np.eye(3)

def project(R: NDArray, t: NDArray, K: NDArray, points_3d: NDArray) -> NDArray:
    """
    This function projects 3D points onto the image plane.
    """
    # Compute the projection matrix
    P = K @ np.hstack([R, t.reshape(-1, 1)])
    
    # Convert the 3D points to homogeneous coordinates
    points_3d_h = np.hstack([points_3d, np.ones((points_3d.shape[0], 1))])
    
    # Project the 3D points onto the image plane
    points_2d = P @ points_3d_h.T
    points_2d = points_2d.T
    points_2d = points_2d[:, :2] / points_2d[:, 2:]
    
    return points_2d

# Define the optimization objective function
def f(x: NDArray, points_3d, points_2d, K=K):
    """
    This function computes the difference between the predicted and observed 2D points.
    """
    # Unpack the angle-axis vector and translation vector
    theta = x[0]
    axis = x[1:3]
    t = x[3:]
    
    # Compute the rotation matrix
    R = o3d.geometry.get_rotation_matrix_from_axis_angle(theta*axis)
    
    # Project the 3D points onto the image plane
    points_2d_pred = project(R, t, K, points_3d)
    
    # Compute the difference between the predicted and observed 2D points
    diff = points_2d_pred - points_2d
    diff = diff.reshape(-1)
    return diff